In [1]:
import pandas as pd
import numpy as np
#실제 그래프를 그릴 수 있는 서브 함수
import matplotlib.pyplot as plt
import seaborn as sns

#경고표시 생략(일시적으로 생략하는게 좋음) 
import warnings
warnings.filterwarnings(action='ignore')

#한글폰트 지정 
import matplotlib.font_manager as fm
font_name= fm.FontProperties(fname="C:/Windows/Fonts/malgun.ttf").get_name()
plt.rc("font",family=font_name)

import matplotlib as mpl
mpl.rcParams['axes.unicode_minus']=False

# 머신러닝을 위한 모듈 
from sklearn.model_selection import train_test_split

import tensorflow as tf

In [ ]:
######## 더 간단하게 쓰기 --> keras

In [2]:
# 시간의 흐름에 따라 지금 단어와 이전 단어를 알아야 함. 

## 1. 정의

+ 시계열 데이터와 같이 시간의 흐름에 따라 변화하는 데이터를 학습하기 위한 딥러닝 모델 

+ 기준 시점(t)과 다음 시점(t+1)에 네트워크를 연결하여 구성한 인공신경망

## 2. 의의

+ '그는 그녀에게 사과의 의미로 빨간 사과 한 바구니를 건내주며 미안하다고 말했다.'  --> 문장에 대한 전처리가 필요함. 

+ 위의 문장과 같이 단순한 사전적 의미 뿐만 아니라, 앞 뒤 문장의 요소들을 종합적으로 판단하여 의미를 파악하고 해석. 

## 3. 자연어 처리(NLP)

+ Thesaurus : 유의어 사전 --> 비슷한 단어들은 사전으로 만들어 참고, 가장 쉽고 간단한 방법
    - < 단점 >
    - 시대변화에 대응하기가 어렵다. 
    - 단어의 미묘한 차이를 표현하기 어렵 --> 융통성을 발휘하기 힘듦. 
    
+ 통계 기반 기법
    - 어떤 단어가 주목했을 때 그 주변에 어떤 단어가 몇번 등장했는지를 세어 집계하는 방법 
    - 데이터가 많이 커짐 --> 거대 행렬 발생

+ 추론 기반 기법 : Word2Vec -> 단어를 벡터화 시키겠다(행렬로)
    - 유사도 측정가능 
    - gensin -- > 유사도 측정
    - 엄청난 데이터가 필요함 
    
## 4. 두가지 결과 

+ 다중 분류 --> 위쪽으로 분류 
    - 품사 분류 
        e.g. I google at work 나는 일할 때 구글링한다.
             I work at google 나는 구글에서 일한다. 
             --> 품사를 알아야 구분 가능 
+ 이진 분류 
    - 감정 분석할 때 - 화났냐 안났냐 
        e.g. traffic tiket fines 
             traffic is fine 
             
## 5. 종류

+ Vanilla RNN
+ LSTM --> 실제로 많이 쓰임 
    - tanh - 탄젠트 h 
    - 이전 기억을 계속 곱하는 

## 6. 실습(Vanilla RNN)

### (1) 단일 cell을 통한 계산 처리 과정: 기본 사용법

In [8]:
##### 입력 데이터 : 1 행 2열 

inputs= np.array([[[1,2]]])

In [9]:
### 단일cell
import warnings
warnings.filterwarnings(action='ignore')

tf.reset_default_graph()
tf.set_random_seed(777) # 난수 값 고정 

tf_inputs = tf.constant(inputs,dtype=tf.float32)

##### 출력 데이터 : 1 행 3열 
rnn_cell=tf.nn.rnn_cell.BasicRNNCell(num_units=3) # --> 히든 계층을 부른 박스 
# 동작을 시켜야 함 정적이든 동적이든 
outputs,state=tf.nn.dynamic_rnn(cell=rnn_cell,dtype=tf.float32,inputs=tf_inputs)# 두개의 결과 값이 나옴
print(outputs)
print(state)

print('\n Weights')
for w in tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES):
    print(w)

print('---------------------------------------------------------')

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    outputs_run,state_run = sess.run([outputs,state])
    
    print('output values')
    print(outputs_run)
    
    print('Wnstate value')
    print(state_run)
    
    print('WnWeights')
    variable_names = [v.name for v in tf.trainable_variables()]
    values = sess.run(variable_names)
    for k,v in zip(variable_names,values):
        print(k,v)



Instructions for updating:
This class is equivalent as tf.keras.layers.SimpleRNNCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
Instructions for updating:
Please use `layer.add_weight` method instead.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Tensor("rnn/transpose_1:0", shape=(1, 1, 3), dtype=float32)
Tensor("rnn/while/Exit_3:0", shape=(1, 3), dtype=float32)

 Weights


<tf.Variable 'rnn/basic_rnn_cell/kernel:0' shape=(5, 3) dtype=float32_ref>
<tf.Variable 'rnn/basic_rnn_cell/bias:0' shape=(3,) dtype=float32_ref>
---------------------------------------------------------


output values
[[[-0.9314169   0.75578666 -0.6819246 ]]]
Wnstate value
[[-0.9314169   0.75578666 -0.6819246 ]]
WnWeights

rnn/basic_rnn_cell/kernel:0 [[-0.62831575  0.38538355  0.79733914]
 [-0.5203329   0.30046564 -0.8150209 ]
 [ 0.39399797 

### (2) 다중 cell 사용 

In [2]:
# I work at google = [[1,0,0,0],[0,1,0,0],[0,0,1,0],[0,0,0,1]]
# I google at work = [[1,0,0,0],[0,0,0,1],[0,0,1,0],[0,1,0,0]]

inputs = np.array([
    [[1,0,0,0],[0,1,0,0],[0,0,1,0],[0,0,0,1]],
    [[1,0,0,0],[0,0,0,1],[0,0,1,0],[0,1,0,0]]
])



In [3]:
import warnings
warnings.filterwarnings(action='ignore')

tf.reset_default_graph()
tf.set_random_seed(777) # 난수 값 고정 

tf_inputs = tf.constant(inputs,dtype=tf.float32)

##### 출력 데이터 : 1 행 3열 
rnn_cell=tf.nn.rnn_cell.BasicRNNCell(num_units=3) # --> 히든 계층을 부른 박스 
# 동작을 시켜야 함 정적이든 동적이든 
outputs,state=tf.nn.dynamic_rnn(cell=rnn_cell,dtype=tf.float32,inputs=tf_inputs)# 두개의 결과 값이 나옴

print(outputs)
print('---------------------------------------------------------')
print(state) # 입력의 갯수, 출력의 갯수

print('\n Weights')
for w in tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES):
    print(w)

print('---------------------------------------------------------')





Instructions for updating:
This class is equivalent as tf.keras.layers.SimpleRNNCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
Instructions for updating:
Please use `layer.add_weight` method instead.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Tensor("rnn/transpose_1:0", shape=(2, 4, 3), dtype=float32)
---------------------------------------------------------
Tensor("rnn/while/Exit_3:0", shape=(2, 3), dtype=float32)

 Weights


<tf.Variable 'rnn/basic_rnn_cell/kernel:0' shape=(7, 3) dtype=float32_ref>
<tf.Variable 'rnn/basic_rnn_cell/bias:0' shape=(3,) dtype=float32_ref>
---------------------------------------------------------


In [12]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    outputs_run,state_run = sess.run([outputs,state])
    
    print('output values')
    print(outputs_run)
    
    print('Wnstate value')
    print(state_run)
    
    print('WnWeights')
    variable_names = [v.name for v in tf.trainable_variables()]
    values = sess.run(variable_names)
    for k,v in zip(variable_names,values):
        print(k,v)

output values
[[[-0.50944704  0.33166462  0.6126557 ]
  [-0.20793891  0.24406303 -0.75278705]
  [-0.06346128 -0.52844936  0.68356085]
  [-0.36491966  0.8857268  -0.02324395]]

 [[-0.50944704  0.33166462  0.6126557 ]
  [-0.30707452  0.62735885  0.21719742]
  [ 0.5043804  -0.14038289  0.3744523 ]
  [-0.11641277  0.70696247 -0.7512605 ]]]
Wnstate value
[[-0.36491966  0.8857268  -0.02324395]
 [-0.11641277  0.70696247 -0.7512605 ]]
WnWeights
rnn/basic_rnn_cell/kernel:0 [[-0.56198275  0.34469748  0.7131618 ]
 [-0.4653999   0.2687447  -0.7289769 ]
 [ 0.35240245  0.14910203  0.36339748]
 [-0.57169586  0.7567036   0.47109187]
 [ 0.3678255   0.5934322  -0.01885086]
 [ 0.31208777 -0.40880746  0.22867584]
 [ 0.5521256   0.682691   -0.5481483 ]]
rnn/basic_rnn_cell/bias:0 [0. 0. 0.]


### (3) 'hihello'문장 연습

In [13]:
tf.reset_default_graph()

idx2char = ['h', 'i', 'e', 'l', 'o']

x_one_hot = [[
    [1, 0, 0, 0, 0],  # h 0
    [0, 1, 0, 0, 0],  # i 1
    [1, 0, 0, 0, 0],  # h 0
    [0, 0, 1, 0, 0],  # e 2
    [0, 0, 0, 1, 0],  # l 3
    [0, 0, 0, 1, 0]   # l 3
]]

y_data = [[1, 0, 2, 3, 3, 4]] 

In [14]:
##### parameter
sequence_len = 6
num_classes = 5

In [15]:
##### 입출력 준비 
tf.reset_default_graph()

X = tf.placeholder(tf.float32, shape=[None, sequence_len, num_classes])
y = tf.placeholder(tf.int32, shape=[None, sequence_len])

In [16]:
##### rnn 모델 
cell = tf.contrib.rnn.BasicRNNCell(num_units=num_classes)
outputs, state = tf.nn.dynamic_rnn(cell=cell, inputs=X, dtype=tf.float32)
print(outputs)
print(state)


The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Tensor("rnn/transpose_1:0", shape=(?, 6, 5), dtype=float32)
Tensor("rnn/while/Exit_3:0", shape=(?, 5), dtype=float32)


In [17]:
##### 평면화(Flat Layer)
X_for_fc = tf.reshape(outputs,[-1,num_classes])
print(X_for_fc)

Tensor("Reshape:0", shape=(?, 5), dtype=float32)


In [18]:
##### FC Layer

outputs = tf.contrib.layers.fully_connected(inputs=X_for_fc, num_outputs=10, activation_fn=None)
outputs = tf.contrib.layers.fully_connected(inputs=outputs, num_outputs=num_classes, activation_fn=None)
#print(outputs)

outputs = tf.reshape(outputs, [1, sequence_len, num_classes])
W = tf.ones([1, sequence_len])
cost = tf.reduce_mean(tf.contrib.seq2seq.sequence_loss(logits=outputs, targets=y, weights=W))
train = tf.train.AdamOptimizer(learning_rate=0.1).minimize(cost)



Instructions for updating:
Please use `layer.__call__` method instead.



In [19]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    for i in range(10):
        _,c=sess.run([train,cost],feed_dict={X:x_one_hot,y:y_data})
        preds=sess.run(tf.arg_max(outputs,2),feed_dict={X:x_one_hot})
        print(i,c,'예측값 :',preds,'정답 :',y_data)
        
    ##### 숫자로 된 것을 문자로 다시 복원해서 
    result = [idx2char[c] for c in np.squeeze(preds)]
    print('결과:',','.join(result))
    


0 1.4986173 예측값 : [[3 3 2 3 3 3]] 정답 : [[1, 0, 2, 3, 3, 4]]
1 1.0085273 예측값 : [[1 0 2 3 3 3]] 정답 : [[1, 0, 2, 3, 3, 4]]
2 0.6378594 예측값 : [[1 0 2 3 3 1]] 정답 : [[1, 0, 2, 3, 3, 4]]
3 0.4264063 예측값 : [[1 0 2 3 3 4]] 정답 : [[1, 0, 2, 3, 3, 4]]
4 0.2296506 예측값 : [[1 0 2 3 3 4]] 정답 : [[1, 0, 2, 3, 3, 4]]
5 0.13723554 예측값 : [[1 0 2 3 3 4]] 정답 : [[1, 0, 2, 3, 3, 4]]
6 0.08035306 예측값 : [[1 0 2 3 3 4]] 정답 : [[1, 0, 2, 3, 3, 4]]
7 0.038433924 예측값 : [[1 0 2 3 3 4]] 정답 : [[1, 0, 2, 3, 3, 4]]
8 0.016992657 예측값 : [[1 0 2 3 3 4]] 정답 : [[1, 0, 2, 3, 3, 4]]
9 0.009578231 예측값 : [[1 0 2 3 3 4]] 정답 : [[1, 0, 2, 3, 3, 4]]
결과: i,h,e,l,l,o


In [20]:
##### 숫자로 된 것을 문자로 다시 복원해서 

### (4) 좀 더 긴 문자으로 실습

In [34]:
# sample = ' You will have to submit to your fate wheather you will or not'
sample = ' You will have to '

# 단어 사전 부터 만들기 
idx2char = list(set(sample))
print(idx2char)

# 문자를 숫자로 
char2idx = {c:i for i,c in enumerate(idx2char)}
print(char2idx)

# 숫자로만 
sample_idx = [char2idx[c] for c in sample ]
print(sample_idx)

# 입력 데이터 
X_data = [sample_idx[:-1]]
y_data = [sample_idx[1:]]

['l', 'a', 'w', 'o', 'u', 't', 'i', 'h', ' ', 'Y', 'v', 'e']
{'l': 0, 'a': 1, 'w': 2, 'o': 3, 'u': 4, 't': 5, 'i': 6, 'h': 7, ' ': 8, 'Y': 9, 'v': 10, 'e': 11}
[8, 9, 3, 4, 8, 2, 6, 0, 0, 8, 7, 1, 10, 11, 8, 5, 3, 8]


In [35]:
#####parameter
sequence_len = len(sample)-1
num_classes = len(idx2char)

In [36]:
##### 입출력준비

tf.reset_default_graph()

X= tf.placeholder(tf.int32, shape=[None, sequence_len])
y= tf.placeholder(tf.int32, shape=[None, sequence_len])

x_one_hot=tf.one_hot(X,num_classes)
# 텐서플로 one_hot은 차원을 늘려줌 



In [37]:
# 모델 

cell = tf.contrib.rnn.BasicRNNCell(num_units=num_classes)
outputs, state = tf.nn.dynamic_rnn(cell=cell, inputs=x_one_hot, dtype=tf.float32)
print(outputs)
print(state)

Tensor("rnn/transpose_1:0", shape=(?, 17, 12), dtype=float32)
Tensor("rnn/while/Exit_3:0", shape=(?, 12), dtype=float32)


In [38]:
W = tf.ones([1, sequence_len])
cost = tf.reduce_mean(tf.contrib.seq2seq.sequence_loss(logits=outputs, targets=y, weights=W))
train = tf.train.AdamOptimizer(learning_rate=0.1).minimize(cost)

In [39]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    for i in range(500):
        _,c=sess.run([train,cost],feed_dict={X:X_data,y:y_data})
        preds=sess.run(tf.arg_max(outputs,2),feed_dict={X:X_data})
        
        if i %100==0:
            print(i,'cost:',c)                      
#             print('예측값 :',preds)
#             print('정답 :',y_data)
            print('--------------------------------------')
    
    
       
    ##### 숫자로 된 것을 문자로 다시 복원해서 
    result = [idx2char[c] for c in np.squeeze(preds)]
    print('결과:',','.join(result))
    

0 cost: 2.4575086
--------------------------------------
100 cost: 0.958887
--------------------------------------
200 cost: 0.94848007
--------------------------------------
300 cost: 0.9444304
--------------------------------------
400 cost: 0.94216704
--------------------------------------
결과: Y,o,u, ,w,i,l, , ,h,a,v,e, ,t,o, 


## 7. LSTM(Long Short Trerm Memory)

+ BPTT : Back Propagation Through Time 
    - Gradient Vanishing 
    - Gradient Exploding
    
+ http://colah.github.io/posts/2015-08-Understanding-LSTMs/
    

In [43]:
tf.reset_default_graph()
tf.set_random_seed(777)

inputs= np.array([[[1,0]]])

tf_inputs =tf.constant(inputs,dtype=tf.float32)
cell = tf.nn.rnn_cell.BasicLSTMCell(num_units=1)
outputs,states=tf.nn.dynamic_rnn(cell=cell,dtype=tf.float32,inputs=tf_inputs)

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    outputs_run,state_run=sess.run([outputs,states])
    
    print(outputs_run)
    print(state_run.h)
    print(state_run.c)
    

[[[0.09927537]]]
[[0.09927537]]
[[0.18134572]]


### (1) 실습1 

In [4]:
##### 데이터 준비

paragraph_dict_list = [
         {'paragraph': 'dishplace is located in sunnyvale downtown there is parking around the area but it can be difficult to find during peak business hours my sisters and i came to this place for dinner on a weekday they were really busy so i highly recommended making reservations unless you have the patience to wait', 'category': 'food'},
         {'paragraph': 'service can be slower during busy hours but our waiter was courteous and help gave some great entree recommendations', 'category': 'food'},
         {'paragraph': 'portions are huge both french toast and their various omelettes are really good their french toast is probably 1.5x more than other brunch places great place to visit if you are hungry and dont want to wait 1 hour for a table', 'category': 'food'},
         {'paragraph': 'we started with apps going the chicken and waffle slides and chicken nachos the sliders were amazing and the nachos were good too maybe by themselves the nachos would have scored better but after those sliders they were up against some tough competition', 'category': 'food'},
         {'paragraph': 'the biscuits and gravy was too salty two people in my group had the gravy and all thought it was too salty my hubby ordered a side of double egg and it was served on two small plates who serves eggs to one person on separate plates we commented on that when it was delivered and even the server laughed and said she doesnt know why the kitchen does that presentation of food is important and they really missed on this one', 'category': 'food'},
         {'paragraph': 'the garlic fries were a great starter (and a happy hour special) the pancakes looked and tasted great and were a fairly generous portion', 'category': 'food'},
         {'paragraph': 'our meal was excellent i had the pasta ai formaggi which was so rich i didnt dare eat it all although i certainly wanted to excellent flavors with a great texture contrast between the soft pasta and the crisp bread crumbs too much sauce for me but a wonderful dish', 'category': 'food'},
         {'paragraph': 'what i enjoy most about palo alto is so many restaurants have dog-friendly seating outside i had bookmarked italico from when they first opened about a 1.5 years ago and was jonesing for some pasta so time to finally knock that bookmark off', 'category': 'food'},
         {'paragraph': 'the drinks came out fairly quickly a good two to three minutes after the orders were taken i expected my iced tea to taste a bit more sweet but this was straight up green tea with ice in it not to complain of course but i was pleasantly surprised', 'category': 'food'},
         {'paragraph': 'despite the not so good burger the service was so slow the restaurant wasnt even half full and they took very long from the moment we got seated to the time we left it was almost 2 hours we thought that it would be quick since we ordered as soon as we sat down my coworkers did seem to enjoy their beef burgers for those who eat beef however i will not be returning it is too expensive and extremely slow service', 'category': 'food'},
    
         {'paragraph': 'the four reigning major champions simona halep caroline wozniacki angelique kerber and defending us open champion sloane stephens could make a case for being the quartet most likely to succeed especially as all but stephens has also enjoyed the no1 ranking within the last 14 months as they prepare for their gruelling new york campaigns they currently hold the top four places in the ranks', 'category': 'sports'},
         {'paragraph': 'the briton was seeded nn7 here last year before a slump in form and confidence took her down to no46 after five first-round losses but there have been signs of a turnaround including a victory over a sub-par serena williams in san jose plus wins against jelena ostapenko and victoria azarenka in montreal. konta pulled out of new haven this week with illness but will hope for good things where she first scored wins in a major before her big breakthroughs to the semis in australia and wimbledon', 'category': 'sports'},
         {'paragraph': 'stephens surged her way back from injury in stunning style to win her first major here last year—and ranked just no83 she has since proved what a big time player she is winning the miami title via four fellow major champions then reaching the final at the french open back on north american hard courts she ran to the final in montreal only just edged out by halep she has also avoided many of the big names in her quarter—except for wild card azarenka as a possible in the third round', 'category': 'sports'},
         {'paragraph': 'when it came to england chances in the world cup it would be fair to say that most fans had never been more pessimistic than they were this year after enduring years of truly dismal performances at major tournaments – culminating in the 2014 event where they failed to win any of their three group games and finished in bottom spot those results led to the resignation of manager roy hodgson', 'category': 'sports'},
         {'paragraph': 'the team that eliminated russia – croatia – also improved enormously during the tournament before it began their odds were 33/1 but they played with real flair and star players like luka modric ivan rakitic and ivan perisic showed their quality on the world stage having displayed their potential by winning all three of their group stage games croatia went on to face difficult tests like the semi-final against england', 'category': 'sports'},
         {'paragraph': 'the perseyside outfit finished in fourth place in the premier league table and without a trophy last term after having reached the champions league final before losing to real madrid', 'category': 'sports'},
         {'paragraph': 'liverpool fc will return to premier league action on saturday lunchtime when they travel to leicester city in the top flight as they look to make it four wins in a row in the league', 'category': 'sports'},
         {'paragraph': 'alisson signed for liverpool fc from as roma this summer and the brazilian goalkeeper has helped the reds to keep three clean sheets in their first three premier league games', 'category': 'sports'},
         {'paragraph': 'but the rankings during that run-in to new york hid some very different undercurrents for murray had struggled with a hip injury since the clay swing and had not played a match since losing his quarter-final at wimbledon and he would pull out of the us open just two days before the tournament began—too late however to promote nederer to the no2 seeding', 'category': 'sports'},
         {'paragraph': 'then came the oh-so-familiar djokovic-nadal no-quarter-given battle for dominance in the thiadal more than once pulled off a reverse smash and had his chance to seal the tie-break but it was djokovic serving at 10-9 who dragged one decisive error from nadal for a two-sets lead', 'category': 'sports'}
]
    
#     스포츠와 관련인가 아닌가 구분 
df=pd.DataFrame(paragraph_dict_list)
df=df[['paragraph','category']]
df

,paragraph,category
0,dishplace is located in sunnyvale downtown the...,food
1,service can be slower during busy hours but ou...,food
2,portions are huge both french toast and their ...,food
3,we started with apps going the chicken and waf...,food
4,the biscuits and gravy was too salty two peopl...,food
5,the garlic fries were a great starter (and a h...,food
6,our meal was excellent i had the pasta ai form...,food
7,what i enjoy most about palo alto is so many r...,food
8,the drinks came out fairly quickly a good two ...,food
9,despite the not so good burger the service was...,food


#### 1) Preprocessing

In [5]:
##### 단어를 분리한뒤 중복된 단어를 제거 
word_list = []

for data in df["paragraph"]:
    word_list.append(data.split(" "))
    
word = []
for i in word_list:
    for j in i:
        word.append(j)
        
print(len(set(word)))
print(word)


530
['dishplace', 'is', 'located', 'in', 'sunnyvale', 'downtown', 'there', 'is', 'parking', 'around', 'the', 'area', 'but', 'it', 'can', 'be', 'difficult', 'to', 'find', 'during', 'peak', 'business', 'hours', 'my', 'sisters', 'and', 'i', 'came', 'to', 'this', 'place', 'for', 'dinner', 'on', 'a', 'weekday', 'they', 'were', 'really', 'busy', 'so', 'i', 'highly', 'recommended', 'making', 'reservations', 'unless', 'you', 'have', 'the', 'patience', 'to', 'wait', 'service', 'can', 'be', 'slower', 'during', 'busy', 'hours', 'but', 'our', 'waiter', 'was', 'courteous', 'and', 'help', 'gave', 'some', 'great', 'entree', 'recommendations', 'portions', 'are', 'huge', 'both', 'french', 'toast', 'and', 'their', 'various', 'omelettes', 'are', 'really', 'good', 'their', 'french', 'toast', 'is', 'probably', '1.5x', 'more', 'than', 'other', 'brunch', 'places', 'great', 'place', 'to', 'visit', 'if', 'you', 'are', 'hungry', 'and', 'dont', 'want', 'to', 'wait', '1', 'hour', 'for', 'a', 'table', 'we', 'start

In [16]:
results= set()
df['paragraph']
# split은 문자열 --> 문자열로 바꿔줘야 스플릿 가능 
df['paragraph'].str.split('').apply(results.update)

0     None
1     None
2     None
3     None
4     None
5     None
6     None
7     None
8     None
9     None
10    None
11    None
12    None
13    None
14    None
15    None
16    None
17    None
18    None
19    None
Name: paragraph, dtype: object

In [17]:
# 단어와 숫자를 매칭해서 dict 자료형 변수 작성(idx2word)
idx2word= dict(enumerate(results))
idx2word

{0: '',
 1: 'f',
 2: 'v',
 3: '8',
 4: 'b',
 5: '.',
 6: '/',
 7: 'c',
 8: ')',
 9: 'h',
 10: 's',
 11: '–',
 12: '3',
 13: 'r',
 14: 'u',
 15: '7',
 16: '4',
 17: 'w',
 18: 'n',
 19: '9',
 20: 'x',
 21: '—',
 22: 'q',
 23: 'd',
 24: 'm',
 25: '5',
 26: ' ',
 27: 'k',
 28: '1',
 29: 'g',
 30: '(',
 31: 'l',
 32: 'e',
 33: 'a',
 34: 'y',
 35: 'j',
 36: '2',
 37: 'z',
 38: 'p',
 39: '0',
 40: 'o',
 41: 't',
 42: '6',
 43: 'i',
 44: '-'}

In [ ]:
word2idx=

In [18]:
word2idx = {word:num for word, num in zip(unique_words, range(len(unique_words)))}
word2idx

NameError: name 'unique_words' is not defined

In [19]:
word2idx = {v:k for k,v in idx2word.items()}

In [20]:
word2idx

{'': 0,
 'f': 1,
 'v': 2,
 '8': 3,
 'b': 4,
 '.': 5,
 '/': 6,
 'c': 7,
 ')': 8,
 'h': 9,
 's': 10,
 '–': 11,
 '3': 12,
 'r': 13,
 'u': 14,
 '7': 15,
 '4': 16,
 'w': 17,
 'n': 18,
 '9': 19,
 'x': 20,
 '—': 21,
 'q': 22,
 'd': 23,
 'm': 24,
 '5': 25,
 ' ': 26,
 'k': 27,
 '1': 28,
 'g': 29,
 '(': 30,
 'l': 31,
 'e': 32,
 'a': 33,
 'y': 34,
 'j': 35,
 '2': 36,
 'z': 37,
 'p': 38,
 '0': 39,
 'o': 40,
 't': 41,
 '6': 42,
 'i': 43,
 '-': 44}

In [30]:
##### 데이터들을 숫자로 변환 

def encoded_paragraph(param):
    words = param.split()
    encoded = []
    
    for w in words: 
        encoded.append([word2idx[w]])
        
    return encoded

def encoded_catagory(param):
    if param == 'food':
        return [1,0]
    else:
        return [0,1]
    
# 각 문장에서 단어 갯수 파악 
def word_cnt(param):
    return len(param.split())

In [31]:
df['enc_paragraph']=df.paragraph.apply(encoded_paragraph)

KeyError: 'dishplace'

In [32]:
df['enc_catagory']=df.paragraph.apply(encoded_catagory)
df.head

<bound method NDFrame.head of                                             paragraph category enc_paragraph  \
0   dishplace is located in sunnyvale downtown the...     food          None   
1   service can be slower during busy hours but ou...     food          None   
2   portions are huge both french toast and their ...     food          None   
3   we started with apps going the chicken and waf...     food          None   
4   the biscuits and gravy was too salty two peopl...     food          None   
5   the garlic fries were a great starter (and a h...     food          None   
6   our meal was excellent i had the pasta ai form...     food          None   
7   what i enjoy most about palo alto is so many r...     food          None   
8   the drinks came out fairly quickly a good two ...     food          None   
9   despite the not so good burger the service was...     food          None   
10  the four reigning major champions simona halep...   sports          None   
11  the br

In [34]:
df['seq_length'] = df.paragraph.apply(word_cnt)
df.head()

,paragraph,category,enc_paragraph,enc_catagory,seq_length
0,dishplace is located in sunnyvale downtown the...,food,None,"[0, 1]",53
1,service can be slower during busy hours but ou...,food,None,"[0, 1]",19
2,portions are huge both french toast and their ...,food,None,"[0, 1]",42
3,we started with apps going the chicken and waf...,food,None,"[0, 1]",43
4,the biscuits and gravy was too salty two peopl...,food,None,"[0, 1]",82


In [35]:
##### 문장의 최대길이 
max_word_cnt= 0 
for row in df['paragraph']:
    if len(row.split())>max_word_cnt:
        max_word_cnt = len(row.split())
        
print(max_word_cnt)

91


In [36]:
max_word_cnt = df['seq_length'].max() 


In [37]:
##### 문장의 길이가 91개가 안될 경우 -1로 채운다

def sequence_padding(enc_param):
    seq_len=len(enc_param)
    
    for i in range(seq_len,max_word_cnt):
        enc_param.append([-1])
        
    return enc_param

In [38]:
df['enc_paragraph']=df.enc_paragraph.apply(sequence_padding)

TypeError: object of type 'NoneType' has no len()

In [ ]:
print(df['enc_paragraph'][0])

In [39]:
enc_paragraph = np.array(df.enc_paragraph.tolist())
encoded_catagory = np.array(df.enc_catagory.tolist())
seq_length = np.array(df.seq_length.tolist())

#### 2) 모델구축 

In [40]:
train_X=enc_paragraph
train_y= enc_category

print(train_X.shape,train_y.shape)

NameError: name 'enc_category' is not defined

In [ ]:
tf.reset_default_graph()
tf.set_random_seed(777)


lr = 0.001
n_epochs=300

X = tf.palceholder(tf.float32,[None,max_word_cnt,1])
y =  tf.palceholder(tf.float32,[None,2])
cell=tf.nn.rnn_cell.LSTMCell(num_units=64)
tf.nn.dynamic_rnn(cell=cell, inputs=X,dtypes=tf.float32),
sequence_length = seq_length



In [41]:
##### FC Layer 

W1= tf.variable(tf.random_normal([64,32]))
b1 = tf.Variable(tf.random_normal([32]))
layer1 = tf.nn.relu(tf.matmul(state.h,W1)+b1)

W2= tf.variable(tf.random_normal([32,2]))
b2 = tf.Variable(tf.random_normal([2]))
logit = tf.matmul(layer1.h,W2)+b2

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=logit,
                                                                labels=y))
train= tf.train.AdamOptimizer(learning_rate=lr).minimize(cost)
print(layer1)

print(logit)

AttributeError: module 'tensorflow' has no attribute 'variable'

In [ ]:
 ### 간단히 
# W1= tf.variable(tf.random_normal([64,32]))
# b1 = tf.Variable(tf.random_normal([32]))
# layer1 = tf.nn.relu(tf.matmul(state.h,W1)+b1)
layer1=tf.layers.dense(state.h,32) # 입력값과 출력값 

# W2= tf.variable(tf.random_normal([32,2]))
# b2 = tf.Variable(tf.random_normal([2]))
# logit = tf.matmul(layer1.h,W2)+b2
logut=tf.layers.dense(layer1,2)
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=logit,
                                                                labels=y))
train= tf.train.AdamOptimizer(learning_rate=lr).minimize(cost)
print(layer1)

print(logit)

In [46]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    for epoch in range(n_epochs):
        _,c=sess.run([train,cost],feed_dict={X:train_X,y:train_y})
        
        if epoch % 50==0:
            preds = tf.nn.softmax(logit)
            correct_pred=tf.equal(tf.argmax(preds,1),tf.argmax(y,1))
            accuracy=tf.reduce_mean(tf.cast(correct_pred,'float'))
            cur_acc=accuracy.eval({X:train_X,y:train_y})
            print(epoch,':',c,','cur_acc)

SyntaxError: invalid syntax (Temp/ipykernel_33468/2883058631.py, line 12)